In [18]:
import scipy.io as sio
import numpy as np

# Load the data
file_path = '/Users/junaeidshoaib/Desktop/Dessertation/Dataset/Matlab_49/BCICIV_1_mat/BCICIV_calib_ds1a.mat'
data = sio.loadmat(file_path)

# Convert to microvolts
eeg_signals = 0.1 * data['cnt'].astype(np.float64)

# Extract markers and sampling rate
markers = data['mrk']
positions = markers['pos'][0][0].flatten()  # Cue positions
classes = markers['y'][0][0].flatten()  # Cue classes

fs = data['nfo']['fs'][0][0][0][0]  # Sampling rate (e.g., 100 Hz)
print(fs)
print(eeg_signals.shape)

100
(190594, 59)


In [2]:
# Define the time window for extracting trials (in seconds)
fixation_time = 2
cue_time = 4 #2-6 seconds cue 
trial_length = cue_time * fs  # Convert to samples

# Extract trials
trials = []
trial_labels = []

for pos, label in zip(positions, classes):
    # Extract the segment from the EEG signals
    trial = eeg_signals[pos:pos + int(trial_length), :]
    trials.append(trial)
    trial_labels.append(label)

trials = np.array(trials)
trial_labels = np.array(trial_labels)

# Output the shape of the trials array
print("Trials shape:", trials.shape)


Trials shape: (200, 400, 59)


In [3]:
from sklearn.model_selection import train_test_split

# Flatten trials to a 2D array (trials x features) if needed
# We will keep them in 3D format for now: (num_trials, num_samples_per_trial, num_channels)

# Split the data
X_train, X_test, y_train, y_test = train_test_split(trials, trial_labels, test_size=0.3, random_state=42)

print("Training set shape:", X_train.shape)
print("Testing set shape:", X_test.shape)


Training set shape: (140, 400, 59)
Testing set shape: (60, 400, 59)


In [4]:
import numpy as np

# Calculate class distributions in the original dataset
unique, counts = np.unique(trial_labels, return_counts=True)
original_distribution = dict(zip(unique, counts))
print("Original Class Distribution:", original_distribution)

# Calculate class distributions in the training set
unique, counts = np.unique(y_train, return_counts=True)
train_distribution = dict(zip(unique, counts))
print("Training Set Class Distribution:", train_distribution)

# Calculate class distributions in the testing set
unique, counts = np.unique(y_test, return_counts=True)
test_distribution = dict(zip(unique, counts))
print("Testing Set Class Distribution:", test_distribution)


Original Class Distribution: {-1: 100, 1: 100}
Training Set Class Distribution: {-1: 70, 1: 70}
Testing Set Class Distribution: {-1: 30, 1: 30}


In [10]:
from mne.decoding import CSP
from scipy.signal import butter, lfilter

# Define the bandpass filter function with proper normalization
def bandpass_filter(data, lowcut, highcut, fs, order=4):
    nyq = 0.5 * fs  # Nyquist frequency
    low = lowcut / nyq  # Normalize lowcut frequency
    high = highcut / nyq  # Normalize highcut frequency
    
    b, a = butter(order, [low, high], btype='band')
    y = lfilter(b, a, data, axis=1)
    return y

# Define the frequency bands (Alpha and Beta)
bands = [(8, 12), (12, 30)]  # Example: alpha and beta bands

# Apply bandpass filters and CSP for each band
X_train_features = []
X_test_features = []

for low, high in bands:
    # Filter the data
    X_train_filt = np.array([bandpass_filter(trial, low, high, fs) for trial in X_train])
    X_test_filt = np.array([bandpass_filter(trial, low, high, fs) for trial in X_test])
    
    # Apply CSP
    csp = CSP(n_components=59, reg=None, log=True, norm_trace=False)
    X_train_csp = csp.fit_transform(X_train_filt, y_train)
    X_test_csp = csp.transform(X_test_filt)
    
    # Append features
    X_train_features.append(X_train_csp)
    X_test_features.append(X_test_csp)

# Concatenate features from all bands
X_train_features = np.concatenate(X_train_features, axis=1)
X_test_features = np.concatenate(X_test_features, axis=1)

print("Training features shape:", X_train_features.shape)
print("Testing features shape:", X_test_features.shape)


Computing rank from data with rank=None
    Using tolerance 1.3e+03 (2.2e-16 eps * 400 dim * 1.5e+16  max singular value)
    Estimated rank (data): 400
    data: rank 400 computed from 400 data channels with 0 projectors
Reducing data rank from 400 -> 400
Estimating class=-1 covariance using EMPIRICAL
Done.
Estimating class=1 covariance using EMPIRICAL
Done.
Computing rank from data with rank=None
    Using tolerance 3.3e+03 (2.2e-16 eps * 400 dim * 3.7e+16  max singular value)
    Estimated rank (data): 400
    data: rank 400 computed from 400 data channels with 0 projectors
Reducing data rank from 400 -> 400
Estimating class=-1 covariance using EMPIRICAL
Done.
Estimating class=1 covariance using EMPIRICAL
Done.
Training features shape: (140, 118)
Testing features shape: (60, 118)


In [11]:
 from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report


In [12]:
# Define a parameter grid to search over
param_grid = {
    'svc__C': [0.1, 1, 10, 100],  # Regularization parameter
    'svc__kernel': ['linear', 'rbf'],  # Kernel type: linear or RBF (Radial Basis Function)
    'svc__gamma': ['scale', 'auto']  # Kernel coefficient for RBF kernel
}


In [13]:
# Create a pipeline with feature scaling and SVM
svm_pipeline = make_pipeline(StandardScaler(), SVC())

# Set up the GridSearchCV
grid_search = GridSearchCV(svm_pipeline, param_grid, cv=5, n_jobs=-1, verbose=1)

# Fit the model using grid search
grid_search.fit(X_train_features, y_train)

# Get the best parameters and the corresponding model
best_model = grid_search.best_estimator_
print("Best parameters found by grid search:", grid_search.best_params_)


Fitting 5 folds for each of 16 candidates, totalling 80 fits
Best parameters found by grid search: {'svc__C': 0.1, 'svc__gamma': 'scale', 'svc__kernel': 'linear'}


In [14]:
# Predict on the test set using the best model
y_pred = best_model.predict(X_test_features)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print("Test set accuracy with best model:", accuracy)

# Detailed classification report
print("\nClassification Report:\n", classification_report(y_test, y_pred))


Test set accuracy with best model: 0.5333333333333333

Classification Report:
               precision    recall  f1-score   support

          -1       1.00      0.07      0.12        30
           1       0.52      1.00      0.68        30

    accuracy                           0.53        60
   macro avg       0.76      0.53      0.40        60
weighted avg       0.76      0.53      0.40        60

